<a href="https://colab.research.google.com/github/manasvinik/baas/blob/master/Copy_of_Blog_Reddit_Batch_Scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install praw

In [ ]:
import praw
import csv

In [ ]:
# Reddit API credentials
client_id = 'ENTER CLIENT ID HERE'
client_secret = 'ENTER CLIENT SECRET HERE'
user_agent = 'ENTER USER AGENT HERE'


In [ ]:
# Initialize PRAW with your credentials
reddit = praw.Reddit(client_id=client_id,
                     client_secret=client_secret,
                     user_agent=user_agent)

In [ ]:
# Function to add questions from a specific listing
def add_questions_from_listing(subreddit, listing, limit, questions):
    for submission in listing(limit=limit):
        if submission.title.endswith('?'):
            questions.add(submission.title)

In [ ]:
# Function to scrape questions from different listings of a subreddit
def scrape_subreddit(subreddit, limit=1000):
    questions = set()
    add_questions_from_listing(subreddit, reddit.subreddit(subreddit).new, limit, questions)
    add_questions_from_listing(subreddit, reddit.subreddit(subreddit).hot, limit, questions)
    add_questions_from_listing(subreddit, reddit.subreddit(subreddit).top, limit, questions)
    add_questions_from_listing(subreddit, reddit.subreddit(subreddit).controversial, limit, questions)
    return questions

In [ ]:
# Function to read subreddits from a CSV and scrape questions, then compile into one file
def scrape_multiple_subreddits_to_single_file(input_filename, output_filename, limit=1000):
    all_questions = set()
    with open(input_filename, mode='r', newline='', encoding='utf-8') as file:
        reader = csv.reader(file)
        for row in reader:
            subreddit = row[0]
            print(f"Scraping subreddit: {subreddit}")
            questions = scrape_subreddit(subreddit, limit)
            all_questions.update(questions)

    with open(output_filename, mode='w', newline='', encoding='utf-8') as outfile:
        writer = csv.writer(outfile)
        writer.writerow(['Question'])
        writer.writerows([[q] for q in all_questions])
    print(f"Scraped a total of {len(all_questions)} questions into {output_filename}")

input_filename = 'subreddits.csv'  # CSV file containing subreddit names
output_filename = 'combined_reddit_questions.csv'  # Output CSV file name
question_limit = 2000  # Change this to the number of questions you want

scrape_multiple_subreddits_to_single_file(input_filename, output_filename, question_limit)


In [ ]:
!pip install  openai --upgrade

In [ ]:
import csv
import openai

# Set your OpenAI API key
openai.api_key = 'ENTER OPEN AI ID HERE'

In [ ]:
def clean_questions_batch(questions_batch, prompt):
    try:
        # Prefix each question with a unique identifier
        combined_questions = "\n".join([f"Question {i+1}: {q}" for i, q in enumerate(questions_batch)])
        full_prompt = f"{prompt}\n\n{combined_questions}"

        response = openai.completions.create(model="gpt-3.5-turbo-instruct",prompt=full_prompt,
        max_tokens=100 * len(questions_batch) # Adjust token limit as needed
        )

        cleaned_responses = response.choices[0].text.strip().split("\n")

        # Process each response, matching them with their corresponding question
        cleaned_questions = []
        for line in cleaned_responses:
            if line.startswith("Question"):
                cleaned_question = line.split(":", 1)[1].strip() if ':' in line else line
                cleaned_questions.append(cleaned_question)

        return cleaned_questions
    except Exception as e:
        print(f"Error cleaning questions: {e}")
        return questions_batch  # Return original batch in case of error

In [ ]:
def process_and_clean_questions(input_filename, output_filename, batch_size=10):
    with open(input_filename, mode='r', newline='', encoding='utf-8') as infile, \
         open(output_filename, mode='w', newline='', encoding='utf-8') as outfile:

        reader = csv.reader(infile)
        writer = csv.writer(outfile)
        next(reader)  # Skip header
        writer.writerow(['Cleaned Question'])

        all_questions = [row[0] for row in reader]  # Assuming questions are in the first column
        prompt = "Here's a list of questions I've compiled. Could you help me rewrite each one into a more direct and concise question without any additional comments or brackets?"

        for i in range(0, len(all_questions), batch_size):
            batch = all_questions[i:i + batch_size]
            cleaned_questions = clean_questions_batch(batch, prompt)
            writer.writerows([[q] for q in cleaned_questions if q.strip()])

In [ ]:
input_filename = 'combined_reddit_questions.csv'  # CSV file with original questions
output_filename = 'cleaned_reddit_questions.csv'  # Output CSV file name

process_and_clean_questions(input_filename, output_filename)